# 1. Data Preprocessing

In [1]:
import pandas as pd
file = pd.read_csv("X_df.csv", index_col= False)

In [2]:
file.head()

,Unnamed: 0,wait,service,price,"('dim', 'sum')","('fried', 'rice')","('orange', 'chicken')","('chow', 'mein')","('pad', 'thai')","('sour', 'soup')",...,"('egg', 'drop', 'soup')","('beef', 'noodle', 'soup')","('kung', 'pao', 'chicken')","('chicken', 'fried', 'rice')","('pork', 'fried', 'rice')","('black', 'bean', 'sauce')","('shrimp', 'fried', 'rice')","('xiao', 'long', 'bao')","('egg', 'foo', 'young')","('dan', 'dan', 'noodles')"
0,0,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,1,1,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,4,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
del file["Unnamed: 0"]

In [4]:
import numpy as np
data = np.array(file)

In [5]:
file2 = pd.read_csv("./Chinese_review.csv")

In [6]:
#indentify if "dimsum" in the review
for i in range(0,len(data)):
    if "dimsum" in file2["text_token"][i]: data[i][3] = 1

In [7]:
#show the all zero data
index_allzero = []
num = 0
for i in data:
    if not 1 in i:
        index_allzero.append(num)
    num += 1

In [8]:
len(index_allzero)

64351

In [9]:
business_id = file2["business_id"]
business_index = list(file2["business_index"])
Y = file2["stars"]

dataset = []
labelset = []

for i in range(0,4434):
    data_sub = []
    label_sub = []
    num = 0
    for business in business_index:
        if business == i:
            data_sub.append(data[num])
            label_sub.append(Y[num])
        num += 1
    dataset.append(data_sub)
    labelset.append(label_sub)

In [10]:
#22-grams phrases
phrases_2 = [('dim','sum'), ('fried','rice'), ('orange', 'chicken'), ('chow', 'mein'), ('pad', 'thai'), ('sour', 'soup'), ('hot', 'pot'), ('spring', 'rolls'), ('lo', 'mein'), ('bbq', 'pork'), ('mongolian', 'beef'), ('egg', 'roll'), ('wonton', 'soup'), ('soy', 'sauce'), ('ice', 'cream'), ('pork', 'belly'), ('milk', 'tea'), ('fried', 'chicken'),  ('sour', 'chicken'), ('white', 'rice'), ('sesame', 'chicken'), ('chicken', 'wings'), ('peking', 'duck'), ('bubble', 'tea')]

In [11]:
#10 -grams phrases
phrases_3 = [('egg', 'drop', 'soup'), ('beef', 'noodle', 'soup'), ('kung', 'pao', 'chicken'), ('chicken', 'fried', 'rice'), ('pork', 'fried', 'rice'), ('black', 'bean', 'sauce'), ('shrimp', 'fried', 'rice'), ('xiao', 'long', 'bao'), ('egg', 'foo', 'young'), ('dan', 'dan', 'noodles')]

In [12]:
name = ["wait", "service", "price"] + phrases_2 + phrases_3
print(name)

['wait', 'service', 'price', ('dim', 'sum'), ('fried', 'rice'), ('orange', 'chicken'), ('chow', 'mein'), ('pad', 'thai'), ('sour', 'soup'), ('hot', 'pot'), ('spring', 'rolls'), ('lo', 'mein'), ('bbq', 'pork'), ('mongolian', 'beef'), ('egg', 'roll'), ('wonton', 'soup'), ('soy', 'sauce'), ('ice', 'cream'), ('pork', 'belly'), ('milk', 'tea'), ('fried', 'chicken'), ('sour', 'chicken'), ('white', 'rice'), ('sesame', 'chicken'), ('chicken', 'wings'), ('peking', 'duck'), ('bubble', 'tea'), ('egg', 'drop', 'soup'), ('beef', 'noodle', 'soup'), ('kung', 'pao', 'chicken'), ('chicken', 'fried', 'rice'), ('pork', 'fried', 'rice'), ('black', 'bean', 'sauce'), ('shrimp', 'fried', 'rice'), ('xiao', 'long', 'bao'), ('egg', 'foo', 'young'), ('dan', 'dan', 'noodles')]


# 2. Fit lasso regression in different business

In [51]:
# give the result of each business
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoCV

name_high = []
name_low = []
wait = []
service = []
price = []
rate_fix = []
rate_origin = []
labelset_fix = []

for x,y in zip(dataset, labelset):
    
    X = np.array(x)
    Y = np.array(y)
    if len(Y) > 37:
        clf = LinearRegression().fit(X,Y)
    
    elif len(Y)> 5:    
        
        clf = LassoCV(cv = 5, n_jobs= -1).fit(X,Y)
    else:
        clf = LassoCV(cv = len(Y), n_jobs= -1).fit(X,Y)
    coef = clf.coef_
    
    index_max = clf.coef_[3:].argmax()
    name_max = name[index_max+3]
    if clf.coef_[index_max+3] == 0:
        name_max = "NA"
    name_high.append(name_max)
    
    index_min = clf.coef_[3:].argmin()
    name_min = name[index_min+3]
    if clf.coef_[index_min+3] == 0:
        name_min = "NA"
    name_low.append(name_min)
    
    coef[index_min + 3] = 0
    
    if clf.coef_[0] < 0:
        wait.append(1)
        coef[0] = 0 
        # 1 means service needs to be improved
    else:
        wait.append(0)
        
    if clf.coef_[1] < 0:
        service.append(1)
        coef[1] = 0
        # 1 means service needs to be improved
    else:
        service.append(0)
        
    if clf.coef_[2] < 0:
        price.append(1)
        coef[2] = 0
        # 1 means service needs to be improved
    else:
        price.append(0)
    
    score = 0 
    label = []
    for i in x:
        result =  np.dot(coef, i) + clf.intercept_
        label.append(result)
        score += result
    
    labelset_fix.append(label)
    rate_fix.append(score/len(x))
    rate_origin.append(sum(y)/len(y))

In [52]:
from collections import OrderedDict
business_id = list(OrderedDict.fromkeys(business_id))

In [53]:
data = {"business_id":business_id, "high_food": name_high, "low_food" : name_low, "wait":wait, "service" : service, "price": price, "rate_origin": rate_origin, "rate_fix": rate_fix }
df = pd.DataFrame(data)

In [67]:
df.head()

,business_id,high_food,low_food,wait,service,price,rate_origin,rate_fix
0,QXAEGFB4oINsVuTFxEYKFQ,"(mongolian, beef)","(orange, chicken)",1,1,1,2.738806,3.061543
1,e_EMySqP0uwlVZfd8mRaaQ,NA,NA,0,0,0,1.600000,1.600000
2,44YFU284Z3KDEy25QyVoUw,"(pad, thai)","(dim, sum)",1,1,1,3.400000,3.697675
3,X8mtoSxY8whtmbDlj0D3Aw,NA,NA,0,0,0,1.750000,1.750000
4,MhnihE0alud0ereVInSt8Q,"(spring, rolls)","(sour, soup)",1,1,1,3.005128,3.199584


In [26]:
df.to_csv("Output.csv", index= False, encoding="utf-8")

In [56]:
#give a example of the first restaurant Emerald Chinese
from sklearn.linear_model import LassoCV
m = LinearRegression().fit(dataset[0],labelset[0])

In [57]:
m.coef_

array([-6.82976391e-02, -2.21587196e-01, -4.95112815e-01,  7.57657378e-01,
       -4.15282427e-01, -7.35753983e+14,  1.95901739e-01, -3.43352891e+14,
        8.93796966e-01, -6.61846274e+13,  7.87243769e-01, -1.31592760e+14,
        1.23248899e+00,  3.75080739e+13, -4.96540652e+13, -6.49113502e-01,
       -7.25738065e+12,  1.12668940e+13,  1.52989409e+00,  1.97056980e+06,
        0.00000000e+00,  2.29880706e+00,  8.88992411e-01,  0.00000000e+00,
       -1.18008321e-01,  2.08464341e+00,  9.33203861e-01,  0.00000000e+00,
        0.00000000e+00,  5.24209378e-01, -3.24427978e-01,  0.00000000e+00,
        1.68007876e+00,  0.00000000e+00, -1.63460795e+00,  0.00000000e+00,
        0.00000000e+00])

In [58]:
name[np.argmin(m.coef_)]

('orange', 'chicken')

In [59]:
def Count_origin(label):
    score = []
    score.append(label.count(1))
    score.append(label.count(2))
    score.append(label.count(3))
    score.append(label.count(4))
    score.append(label.count(5))
    return score

In [60]:
list_origin = [Count_origin(label) for label in labelset]

In [61]:
df_origin = pd.DataFrame(list_origin, columns=[1,2,3,4,5],index=business_id)

In [63]:
df_origin.head()

,1,2,3,4,5
QXAEGFB4oINsVuTFxEYKFQ,34,22,33,35,10
e_EMySqP0uwlVZfd8mRaaQ,4,0,0,1,0
44YFU284Z3KDEy25QyVoUw,56,25,36,85,83
X8mtoSxY8whtmbDlj0D3Aw,2,1,1,0,0
MhnihE0alud0ereVInSt8Q,59,20,20,53,43


In [64]:
labelset_fix_2 = []
for label in labelset_fix:
    score = []
    for i in label:
        score.append(round(i,0))
    labelset_fix_2.append(score)

In [65]:
list_fix = [Count_origin(label) for label in labelset_fix_2]
df_fix = pd.DataFrame(list_fix, columns=[1,2,3,4,5],index=business_id)

In [66]:
df_fix.head()

,1,2,3,4,5
QXAEGFB4oINsVuTFxEYKFQ,0,43,80,7,4
e_EMySqP0uwlVZfd8mRaaQ,0,5,0,0,0
44YFU284Z3KDEy25QyVoUw,0,4,180,69,29
X8mtoSxY8whtmbDlj0D3Aw,0,4,0,0,0
MhnihE0alud0ereVInSt8Q,1,8,148,30,7
